### Comparing the transcript structures of syntelogs

In [10]:
import pandas as pd
import matplotlib as plt

In [11]:
bed_file_leaf = "/scratch/nadjafn/Atlantic_ASE/nf-potato-ase/tama_test_leaf/leaf_chr01.bed"
bed_file_tuber = "/scratch/nadjafn/Atlantic_ASE/nf-potato-ase/tama_test_tuber/tuber_chr01.bed"
cluster_info_leaf = "/scratch/nadjafn/Atlantic_ASE/nf-potato-ase/tama_test_leaf/leaf_chr01_trans_report.txt"
cluster_info_tuber = "/scratch/nadjafn/Atlantic_ASE/nf-potato-ase/tama_test_tuber/tuber_chr01_trans_report.txt"
allele_info = "/DKED/scratch/nadjafn/Atlantic_ASE/nf-potato-ase/results/04_ALLELECOUNTS/Synt_counts_with_info.tsv"

In [56]:
# read in the bed file
bed_filtered_list = []
bed_list = []
for bed_file, cluster_info in zip([bed_file_leaf, bed_file_tuber], [cluster_info_leaf, cluster_info_tuber]):
    bed = pd.read_csv(bed_file, sep="\t", header=None, names=["haplotype_id", "start", "end", "name", "score", "strand", "thickstart", "thickend", "itemRgb", "blockCount", "blockSizes", "blockStarts"])
    # add a colum with transcript id which is a split of the name after ;
    bed["transcript_id"] = bed["name"].str.split(";").str[1]
    clusters = pd.read_csv(cluster_info, sep="\t", usecols=[0, 1])
    # merge the bed file with the cluster info
    bed = bed.merge(clusters, on="transcript_id", how="left")
    bed_unfiltered = bed
    # Define a helper function to check if any two values in a series are within 80%
    def check_within_threshold(values, threshold=0.8):
        sorted_values = sorted(values, reverse= True)  # Sort the values for comparison
        # drop NaN values
        sorted_values = [x for x in sorted_values if str(x) != 'nan']
        if len(sorted_values) > 1:
            if  sorted_values[1]/sorted_values[0] > threshold:
                return True
        return False

    # Process each haplotype group
    result = []
    for haplotype_id, group in bed.groupby("haplotype_id"):
        if check_within_threshold(group['num_clusters'].values, threshold=0.6):
            # If two counts are within 80%, keep the row with the maximum 'blockCount'
            row = group.loc[group['blockCount'].idxmax()]
        else:
            # Otherwise, keep the row with the maximum 'num_clusters'
            row = group.loc[group['num_clusters'].idxmax()]
        result.append(row)

    # Combine the filtered rows into a new DataFrame
    bed_filtered = pd.DataFrame(result)
    bed_filtered = bed_filtered[bed_filtered["num_clusters"] > 50]
    bed_filtered_list.append(bed_filtered)
    bed_list.append(bed)


In [48]:

bed_list[0][bed_list[0]["haplotype_id"].str.contains("Synt_1354_chr01_3G_x4")]

,haplotype_id,start,end,name,score,strand,thickstart,thickend,itemRgb,blockCount,blockSizes,blockStarts,transcript_id,num_clusters
16297,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,0,4700,G3897;G3897.1,40,+,0,4700,"255,0,0",10,"361,97,92,132,186,164,79,129,128,327","0,575,754,929,1144,1488,1759,3754,4160,4373",G3897.1,3
16298,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,0,5362,G3897;G3897.2,40,+,0,5362,"255,0,0",12,"34,240,97,92,132,186,164,79,129,128,408,453","0,121,575,754,929,1144,1488,1759,3754,4160,437...",G3897.2,87
16299,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,3,3819,G3897;G3897.3,40,+,3,3819,"255,0,0",8,"358,97,92,132,186,164,79,65","0,572,751,926,1141,1485,1756,3751",G3897.3,1
16300,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,3,4284,G3897;G3897.4,40,+,3,4284,"255,0,0",10,"31,240,97,92,132,186,164,79,129,124","0,118,572,751,926,1141,1485,1756,3751,4157",G3897.4,1
16301,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,3,4720,G3897;G3897.5,40,+,3,4720,"255,0,0",10,"358,97,92,132,186,164,79,129,128,347","0,572,751,926,1141,1485,1756,3751,4157,4370",G3897.5,2
16302,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,7,4700,G3897;G3897.6,40,+,7,4700,"255,0,0",11,"27,240,97,92,132,186,164,79,129,128,327","0,114,568,747,922,1137,1481,1752,3747,4153,4366",G3897.6,2
16303,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,7,5191,G3897;G3897.7,40,+,7,5191,"255,0,0",12,"27,240,97,92,132,186,164,79,129,128,408,282","0,114,568,747,922,1137,1481,1752,3747,4153,436...",G3897.7,2
16304,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,7,5346,G3897;G3897.8,40,+,7,5346,"255,0,0",9,"354,97,92,132,186,164,79,105,309","0,568,747,922,1137,1481,1752,3747,5030",G3897.8,1
16305,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,8,1915,G3897;G3897.9,40,+,8,1915,"255,0,0",8,"26,240,97,92,132,186,164,156","0,113,567,746,921,1136,1480,1751",G3897.9,1
16306,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,8,4948,G3897;G3897.10,40,+,8,4948,"255,0,0",12,"26,240,97,92,132,186,164,79,129,128,408,39","0,113,567,746,921,1136,1480,1751,3746,4152,436...",G3897.10,1


In [43]:
bed_list[1][bed_list[1]["haplotype_id"].str.contains("Synt_1354_chr01_3G_x4")]

,haplotype_id,start,end,name,score,strand,thickstart,thickend,itemRgb,blockCount,blockSizes,blockStarts,transcript_id,num_clusters
5773,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,3,5222,G2257;G2257.1,40,+,3,5222,"255,0,0",12,"31,240,97,92,132,186,164,79,129,128,408,313","0,118,572,751,926,1141,1485,1756,3751,4157,437...",G2257.1,3
5774,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,3,5362,G2257;G2257.2,40,+,3,5362,"255,0,0",12,"31,240,97,92,132,186,164,79,129,128,408,453","0,118,572,751,926,1141,1485,1756,3751,4157,437...",G2257.2,29
5775,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,3,5362,G2257;G2257.3,40,+,3,5362,"255,0,0",11,"358,97,92,132,186,164,79,129,128,408,453","0,572,751,926,1141,1485,1756,3751,4157,4370,4906",G2257.3,42
5776,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,55,5362,G2257;G2257.4,40,+,55,5362,"255,0,0",10,"306,97,92,132,186,164,79,129,128,989","0,520,699,874,1089,1433,1704,3699,4105,4318",G2257.4,2
5777,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,56,4522,G2257;G2257.5,40,+,56,4522,"255,0,0",10,"305,97,92,132,186,164,79,129,128,149","0,519,698,873,1088,1432,1703,3698,4104,4317",G2257.5,1
5778,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,60,4677,G2257;G2257.6,40,+,60,4677,"255,0,0",10,"301,97,92,132,186,164,79,129,128,304","0,515,694,869,1084,1428,1699,3694,4100,4313",G2257.6,2
5779,Synt_1354_chr01_3G_x4|hap3_Soltu.DM.01G020170....,65,5113,G2257;G2257.7,40,+,65,5113,"255,0,0",11,"296,97,92,132,186,164,79,129,128,408,204","0,510,689,864,1079,1423,1694,3689,4095,4308,4844",G2257.7,1


In [57]:
# we merge the two dataframes
bed = bed_filtered_list[0].merge(bed_filtered_list[1], on="haplotype_id", how="inner", suffixes=("_leaf", "_tuber"))
# drop the columns that are not needed
bed.drop(columns=["start_leaf", "end_leaf", "start_tuber", "end_tuber", "name_leaf", "name_tuber", "score_leaf", "score_tuber", "strand_leaf", "strand_tuber", "thickstart_leaf", "thickstart_tuber", "thickend_leaf", "thickend_tuber", "itemRgb_leaf", "itemRgb_tuber", "blockSizes_leaf", "blockSizes_tuber", "blockStarts_leaf", "blockStarts_tuber"], inplace=True)
# print the rows that have a difference in the number of blockCounts
print(bed[bed["blockCount_leaf"] != bed["blockCount_tuber"]])

                                         haplotype_id  blockCount_leaf  \
30  Synt_3005_chr01_2G_x4|hap2_Soltu.DM.01G042380....                9   
45  Synt_3364_chr01_1G_x4|hap1_Soltu.DM.01G047280....               10   
48  Synt_3364_chr01_4G_x4|hap4_Soltu.DM.01G047280....               11   

   transcript_id_leaf  num_clusters_leaf  blockCount_tuber  \
30           G3223.12                 61                 8   
45            G1958.5                167                 9   
48            G6467.7                158                 9   

   transcript_id_tuber  num_clusters_tuber  
30             G1853.1                 223  
45             G1133.1                 128  
48             G3818.1                 178  


In [55]:
bed

,haplotype_id,blockCount_leaf,transcript_id_leaf,num_clusters_leaf,blockCount_tuber,transcript_id_tuber,num_clusters_tuber
0,Synt_125_chr01_1G_x4|hap1_Soltu.DM.01G001830.1...,5,G843.2,87,5,G503.1,50
1,Synt_125_chr01_2G_x4|hap2_Soltu.DM.01G001830.1...,5,G2135.1,82,5,G1228.1,33
2,Synt_125_chr01_3G_x4|hap3_Soltu.DM.01G001830.1...,5,G3618.1,62,5,G2099.1,45
3,Synt_125_chr01_4G_x4|hap4_Soltu.DM.01G001830.1...,5,G5199.1,93,5,G3054.1,50
4,Synt_1530_chr01_2G_x3|hap2_Soltu.DM.01G022570....,3,G2537.3,184,3,G1420.1,128
...,...,...,...,...,...,...,...
168,no_syntelog_chr01_3G_x|hap3_Soltu.DM.01G044610...,5,G4729.1,121,5,G2782.1,62
169,no_syntelog_chr01_3G_x|hap3_Soltu.DM.10G013440...,2,G3864.3,219,2,G2236.1,75
170,no_syntelog_chr01_4G_x|hap4_Soltu.DM.01G027390...,13,G5749.1,99,13,G3360.2,31
171,no_syntelog_chr01_4G_x|hap4_Soltu.DM.01G044610...,5,G6342.1,186,5,G3733.1,61


In [14]:
# pivot the allele table
# 1. read the allele table
df = pd.read_csv('/scratch/nadjafn/Atlantic_ASE/output/AlleleFinder/allelefinder/allele.adjusted.txt', sep='\t')
# select the rows that have exactly one entry in 'A', 'B', 'C', 'D' columns. so no , and no missing values
df = df[(df['Allele A'].str.count(',') == 0) & (df['Allele B'].str.count(',') == 0) & (df['Allele C'].str.count(',') == 0) & (df['Allele D'].str.count(',') == 0)]
# reset the index
df = df.reset_index(drop=True)
# Add a id column to the dataframe
df['Synt_id'] = df.index
# extract the chromosome from all the allele columns
df['Chr_a'] = df['Allele A'].str.extract(r'(chr\d+)')
df['Chr_b'] = df['Allele B'].str.extract(r'(chr\d+)')
df['Chr_c'] = df['Allele C'].str.extract(r'(chr\d+)')
df['Chr_d'] = df['Allele D'].str.extract(r'(chr\d+)')
# filter that have the same chromomsome in all the alleles
df = df[(df['Chr_a'] == df['Chr_b']) & (df['Chr_a'] == df['Chr_c']) & (df['Chr_a'] == df['Chr_d'])]
df['Synt_id'] = df['Allele A'].str.extract(r'(Soltu.DM.\d+G\d+)')

# If the first pattern is not found, extract the second pattern and fill in the missing values
df['Synt_id'] = df['Synt_id'].fillna(df['Allele A'].str.extract(r'(Soltu.Atl_v3.01_\d+G\d*)')[0])

# 2. pivot the table so that each row is a single allele
# 2.1. select the columns that we want to pivot
df_synt = df[['Synt_id', 'Allele A', 'Allele B', 'Allele C', 'Allele D']]
# 2.2. rename the columns
df_synt.columns = ['Synt_id', '1', '2', '3', '4']
# 2.3. melt the table
df_synt = pd.melt(df_synt, id_vars=['Synt_id'], value_vars=['1', '2', '3', '4'], var_name='haplotype', value_name='haplotype_id')
# replace Synt_id with a part from haplotype id
# Extract the first pattern
print(df_synt)


                        Synt_id haplotype  \
0      Soltu.Atl_v3.01_1G016780         1   
1      Soltu.Atl_v3.01_1G018980         1   
2            Soltu.DM.10G009940         1   
3      Soltu.Atl_v3.01_1G022210         1   
4      Soltu.Atl_v3.01_1G023000         1   
...                         ...       ...   
44019        Soltu.DM.12G029210         4   
44020        Soltu.DM.12G029220         4   
44021        Soltu.DM.12G029230         4   
44022        Soltu.DM.12G029240         4   
44023        Soltu.DM.12G029300         4   

                                            haplotype_id  
0      Synt_1744_chr01_1G_x4|Soltu.Atl_v3.01_1G016780...  
1      Synt_1970_chr01_1G_x4|Soltu.Atl_v3.01_1G018980...  
2      Synt_20601_chr08_1G_x8|hap1_Soltu.DM.10G009940...  
3      Synt_2300_chr01_1G_x4|Soltu.Atl_v3.01_1G022210...  
4      Synt_2499_chr01_1G_x4|Soltu.Atl_v3.01_1G023000...  
...                                                  ...  
44019  no_syntelog_chr12_4G_x|hap4_Soltu.DM.12

In [15]:
merged = pd.merge(df_synt, bed, on="haplotype_id", how="inner")
# group by synt_id and count the number of unuqie blockCount values
count = merged.groupby("Synt_id")["blockCount"].nunique()
# make a barplot of the count
# count the frequency of the counts
plt = count.value_counts().plot(kind='bar')
plt.set_xlabel("Number of exon numbers per syntelog")  # Correct syntax for x-axis label
plt.set_ylabel("Frequency")  # Optionally, add a y-axis label
plt.set_title("Distribution of Exon Numbers per Syntelog")  # Optionally, add a title
 # Display the plot

KeyError: 'Column not found: blockCount'

In [ ]:
# print where count > 1
print(count[count > 1])


Synt_id
Soltu.DM.01G000050    2
Soltu.DM.01G000120    2
Soltu.DM.01G002580    2
Soltu.DM.01G002690    2
Soltu.DM.01G005580    2
Soltu.DM.01G019020    2
Soltu.DM.01G019780    2
Soltu.DM.01G028100    2
Soltu.DM.01G031330    2
Soltu.DM.01G039510    2
Soltu.DM.01G045870    2
Soltu.DM.01G051500    2
Name: blockCount, dtype: int64


In [ ]:
merged[merged["Synt_id"] == "Soltu.DM.01G051500"]

,Synt_id,haplotype,haplotype_id,start,end,name,score,strand,thickstart,thickend,itemRgb,blockCount,blockSizes,blockStarts,transcript_id,num_clusters
105,Soltu.DM.01G051500,2,Synt_3676_chr01_2G_x4|hap2_Soltu.DM.01G051500....,221,2865,G3033;G3033.2,40,+,221,2865,"255,0,0",3,"651,188,494","0,1024,2150",G3033.2,67.0
217,Soltu.DM.01G051500,4,Synt_3676_chr01_4G_x4|hap4_Soltu.DM.01G051500....,169,2891,G5691;G5691.1,40,+,169,2891,"255,0,0",2,"1264,520","0,2202",G5691.1,94.0


In [ ]:
# print merged where Synt_id == Soltu.DM.01G044600
print(bed_unfiltered[bed_unfiltered["haplotype_id"].str.contains("hap2_Soltu.DM.01G051500")])

                                            haplotype_id  start   end  \
10270  Synt_3676_chr01_2G_x4|hap2_Soltu.DM.01G051500....    200  2777   
10271  Synt_3676_chr01_2G_x4|hap2_Soltu.DM.01G051500....    221  2865   
10272  Synt_3676_chr01_2G_x4|hap2_Soltu.DM.01G051500....    224  2633   
10273  Synt_3676_chr01_2G_x4|hap2_Soltu.DM.01G051500....    236  2902   
10274  Synt_3676_chr01_2G_x4|hap2_Soltu.DM.01G051500....    417  2582   
10275  Synt_3676_chr01_2G_x4|hap2_Soltu.DM.01G051500....    877  2988   

                name  score strand  thickstart  thickend  itemRgb  blockCount  \
10270  G3033;G3033.1     40      +         200      2777  255,0,0           2   
10271  G3033;G3033.2     40      +         221      2865  255,0,0           3   
10272  G3033;G3033.3     40      +         224      2633  255,0,0           2   
10273  G3033;G3033.4     40      +         236      2902  255,0,0           3   
10274  G3033;G3033.5     40      +         417      2582  255,0,0           2   
10